In [58]:
elastic_rods_dir = '../../elastic_rods/python/'
weaving_dir = '../'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
import copy
import vis.fields
import matplotlib.cm as cm
import os

In [59]:

# Reloadable libraries
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [68]:
def generate_polygon_module(poly_size, flat_start = True):
    polygon = []
    for i in range(poly_size):
        theta = 2 * np.pi / poly_size * i
        point = np.array([100 * np.sin(theta), 100 * np.cos(theta), 0])
        polygon.append(point)

    tips = []
    if flat_start:
        for i in range(poly_size):
            mid = (polygon[i-1] + polygon[i]) / 2
            height = la.norm(polygon[i-1] - polygon[i]) * np.sqrt(3) / 2
            tips.append(mid + mid / la.norm(mid) * height)
    else:
        for i in range(poly_size):
            mid = (polygon[i-1] + polygon[i]) / 2
            height = np.array([0, 0, la.norm(polygon[i-1] - polygon[i]) * np.sqrt(3) / 2])
            tips.append(mid + height)
    
    point_list = polygon + tips
    edge_list = [(i + 1, i + 2) for i in range(poly_size - 1)] + [(poly_size, 1)] + [(i + 1, poly_size + i + 2) for i in range(poly_size - 1)]  + [(poly_size, poly_size + 1)] + [(i + 2, poly_size + i + 2) for i in range(poly_size - 1)] + [(1, poly_size + 1)]
    polygon_path = 'polygon'
    if not os.path.exists(polygon_path):
        os.makedirs(polygon_path)

    with open ('{}/{}_gon.obj'.format(polygon_path, poly_size), 'w') as f:
        for point in point_list:
            f.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
        for edge in edge_list:
            f.write('l {} {}\n'.format(edge[0], edge[1]))

In [69]:
for i in range(11)[3:]:
    generate_polygon_module(i)

In [70]:
# Polygon Parameters
poly_size = 5
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 15]
ISO_CS = [4, 4]
MODEL_PATH = 'polygon/{}_gon.obj'.format(poly_size)
MODEL_NAME = "{}_gon".format(poly_size)
SUBDIVISION_RESOLUTION = 10
REGULARIZATION_WEIGHT = 0
opts = elastic_rods.NewtonOptimizerOptions()

In [76]:
def initialize_polygon_linkage(poly_size):
    cam_param = ((-5.547141272807298, -0.4176337718243157, 4.000698835299432),
 (0.11831641535495478, -0.5039921608019906, 0.8556159580721264),
 (0.1111, 0.1865, 0.5316))
    cross_section = [1, 40]
    model_path = 'polygon/{}_gon.obj'.format(poly_size)
    subdivision_res = 20
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    driver=l.centralJoint()
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    jdo = l.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    opts = elastic_rods.NewtonOptimizerOptions()
#     Why does ajusting opts not work?
    elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options = opts)
#     elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options = opts)
#     elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options = opts)

    view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view
    
def save_polygon_linkage(poly_size, linkage):
    MODEL_NAME = "{}_gon".format(poly_size)
    export_linkage_geometry_to_obj(linkage, '{}_straight_linkage.obj'.format(MODEL_NAME))


In [77]:

def get_curvature_scalar_field(linkage):
    curvatures = []
    for i in range(linkage.numSegments()):
        curr_kappas = linkage.segment(i).rod.deformedConfiguration().kappa
        curr_kappas = np.array(curr_kappas)[:, 1]
        curr_kappas[0] = curr_kappas[1]
        curr_kappas[-1] = curr_kappas[-2]
        curr_kappas = [kappa ** 2 for kappa in curr_kappas]
        curvatures.append(curr_kappas)
    curvatures = np.array(curvatures)
    sf = vis.fields.ScalarField(linkage, 0.6 * np.ones_like(curvatures), colormap=cm.Blues, vmin = 0, vmax = 1)
    return sf

with so(): poly_linkage, poly_mod_view = initialize_polygon_linkage(poly_size = 6)
poly_mod_view.update(scalarField=get_curvature_scalar_field(poly_linkage))

In [78]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [95]:
@interact(poly_size=widgets.IntSlider(min=3, max=10, step=1, value=3))
def update_polygon_linkage(poly_size):
    global new_poly_mod_view
    cross_section = [1, 40]
    model_path = 'polygon/{}_gon.obj'.format(poly_size)
    subdivision_res = 20
    poly_linkage = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    driver=poly_linkage.centralJoint()
    poly_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    jdo = poly_linkage.dofOffsetForJoint(0)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint    
    #with so(): 
    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 10000
    #opts.beta = 1e-8
    with so():
        elastic_rods.compute_equilibrium(poly_linkage, fixedVars=fixedVars, options = opts)
#         elastic_rods.compute_equilibrium(poly_linkage, fixedVars=fixedVars)
#         elastic_rods.compute_equilibrium(poly_linkage, fixedVars=fixedVars)
    new_poly_mod_view = linkage_vis.getColoredRodOrientationViewer(poly_linkage, width=1024, height=640, bottomColor=[0.5, 0.5, 0.5], topColor=[1, 0, 0])
    poly_mod_view.update(mesh = new_poly_mod_view.mesh, scalarField=get_curvature_scalar_field(poly_linkage))
    

interactive(children=(IntSlider(value=3, description='poly_size', max=10, min=3), Output()), _dom_classes=('wi…

In [96]:
new_poly_mod_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…